<a href="https://colab.research.google.com/github/Laimo64/COMP0249_24-25/blob/main/Supervised_Autoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip -q pelvis_smalllll.zip

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Encoder (可使用 ResNet 或 U-Net Encoder)
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.conv1 = nn.Conv3d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv3d(32, 64, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv3d(64, 128, kernel_size=3, stride=2, padding=1)
        self.conv4 = nn.Conv3d(128, 256, kernel_size=3, stride=2, padding=1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.relu(self.conv4(x))
        return x

# Decoder (對應 Encoder，進行反卷積重建)
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.deconv1 = nn.ConvTranspose3d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.deconv2 = nn.ConvTranspose3d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.deconv3 = nn.ConvTranspose3d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.deconv4 = nn.ConvTranspose3d(32, 1, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.deconv1(x))
        x = self.relu(self.deconv2(x))
        x = self.relu(self.deconv3(x))
        x = self.sigmoid(self.deconv4(x))  # 最後輸出範圍為 [0,1]
        return x

# 整合 Encoder + Decoder
class SupervisedAutoencoder(nn.Module):
    def __init__(self):
        super(SupervisedAutoencoder, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# 訓練函式
def train(model, dataloader, epochs=50, lr=1e-4):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.L1Loss()  # MAE 損失

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for mri, ct in dataloader:
            mri, ct = mri.to(device), ct.to(device)
            optimizer.zero_grad()
            output = model(mri)
            loss = loss_fn(output, ct)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataloader)}")

# 初始化模型
model = SupervisedAutoencoder()


In [8]:
import os
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import numpy as np
import nibabel as nib  # 用於處理醫學影像 NIfTI 格式

from scipy.ndimage import zoom

def resize_volume(img, target_shape=(128, 128, 128)):
    """將 3D 影像 resize 成固定大小"""
    # 計算每個維度的縮放比例
    scale_factors = [target_shape[i] / img.shape[i] for i in range(3)]
    img_resized = zoom(img, scale_factors, order=1)  # order=1 為線性插值
    return img_resized


class MRCTDataset(Dataset):
    def __init__(self, mri_paths, ct_paths, transform=None):
        self.mri_paths = mri_paths
        self.ct_paths = ct_paths
        self.transform = transform

    def __len__(self):
        return len(self.mri_paths)

    def __getitem__(self, idx):
      mri = nib.load(self.mri_paths[idx]).get_fdata()
      ct = nib.load(self.ct_paths[idx]).get_fdata()

      # **統一大小**
      target_shape = (128, 128, 128)  # 你可以根據數據集決定
      mri = resize_volume(mri, target_shape)
      ct = resize_volume(ct, target_shape)

      # 轉換為 PyTorch Tensor
      mri, ct = torch.tensor(mri, dtype=torch.float32), torch.tensor(ct, dtype=torch.float32)
      mri, ct = mri.unsqueeze(0), ct.unsqueeze(0)  # 增加通道維度 (C, H, W, D)

      if self.transform:
          mri = self.transform(mri)
          ct = self.transform(ct)

      return mri, ct


def get_file_paths(data_root):
    mri_paths, ct_paths = [], []

    for patient in sorted(os.listdir(data_root)):  # 遍歷所有病人資料夾
        patient_dir = os.path.join(data_root, patient)
        if os.path.isdir(patient_dir):
            mri_file = os.path.join(patient_dir, "mr.nii.gz")
            ct_file = os.path.join(patient_dir, "ct.nii.gz")

            if os.path.exists(mri_file) and os.path.exists(ct_file):
                mri_paths.append(mri_file)
                ct_paths.append(ct_file)

    return mri_paths, ct_paths

# 設置 DataLoader
data_root = "/content/pelvis_smalllll"  # 你的數據文件夾
mri_paths, ct_paths = get_file_paths(data_root)  # **讀取 MRI 和 CT 文件路徑**
dataset = MRCTDataset(mri_paths, ct_paths, transform=transforms.Normalize(0.5, 0.5))
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)


# 訓練模型
train(model, dataloader, epochs=50)


Epoch 1/50, Loss: 1093.3494873046875


KeyboardInterrupt: 